In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler


In [3]:
import sys, argparse, logging
import pymssql
import _mssql
from datetime import datetime, time
from decimal import Decimal
import pymongo
from pymongo import MongoClient
from tqdm import tqdm, tnrange, tqdm_notebook
import pandas as pd
from tmqr.settings import *
from tmqrfeed.assetsession import AssetSession
import pickle
import pytz
try:
    from tmqr.settings_local import *
except:
    pass
import pyximport

pyximport.install(setup_args={"include_dirs": np.get_include()})
from tmqrfeed.quotes.compress_daily_ohlcv import compress_daily
import os

from tmqrfeed.assetsession import AssetSession
from tmqrfeed.quotes.dataframegetter import DataFrameGetter
from tmqrfeed.datafeed import DataFeed

In [4]:
feed = DataFeed()

In [5]:
chain = feed.get_fut_chain("US.CL")
chain_values = chain.get_list(datetime(2011, 1, 1))

In [6]:
chain_values    

,date_end,date_start
ticker,,
US.F.CL.G11.110120,2011-01-18,2010-12-17
US.F.CL.H11.110222,2011-02-18,2011-01-18
US.F.CL.J11.110322,2011-03-18,2011-02-18
US.F.CL.K11.110419,2011-04-15,2011-03-18
US.F.CL.M11.110520,2011-05-18,2011-04-15
US.F.CL.N11.110622,2011-06-20,2011-05-18
US.F.CL.Q11.110720,2011-07-18,2011-06-20
US.F.CL.U11.110822,2011-08-18,2011-07-18
US.F.CL.V11.110921,2011-09-19,2011-08-18


In [37]:
fut = chain.get_contract(datetime(2012, 1, 1))
fut2 = chain.get_contract(datetime(2012, 1, 1), offset=1)

In [38]:
series1 = fut.get_series()
series2 = fut2.get_series()

In [13]:
series  = fut.get_series()

In [14]:
series2

,o,h,l,c,v
dt,,,,,
2012-01-17 16:00:00-08:00,101.42,101.45,101.41,101.45,18.0
2012-01-17 16:01:00-08:00,101.43,101.45,101.43,101.44,6.0
2012-01-17 16:02:00-08:00,101.42,101.42,101.42,101.42,1.0
2012-01-17 16:03:00-08:00,101.42,101.42,101.42,101.42,1.0
2012-01-17 16:04:00-08:00,101.42,101.42,101.40,101.40,6.0
2012-01-17 16:05:00-08:00,101.41,101.41,101.41,101.41,2.0
2012-01-17 16:06:00-08:00,101.40,101.43,101.40,101.43,4.0
2012-01-17 16:07:00-08:00,101.43,101.44,101.42,101.44,15.0
2012-01-17 16:08:00-08:00,101.41,101.41,101.40,101.40,2.0


In [39]:
prev_series, position1  = compress_daily(DataFrameGetter(series1), fut)
new_series, position2  = compress_daily(DataFrameGetter(series2), fut2)

In [28]:
series1.to_csv('fut_series_for_contfut1.csv.gz', compression='gzip')

In [29]:
series2.to_csv('fut_series_for_contfut2.csv.gz', compression='gzip')

In [42]:
prev_series

,c,exec,h,l,o,v
dt,,,,,,
2011-12-19 10:40:00-08:00,94.06,94.07,94.63,93.23,93.35,100935.0
2011-12-20 10:40:00-08:00,97.48,97.45,97.61,94.75,94.77,151103.0
2011-12-21 10:40:00-08:00,98.66,98.56,99.25,96.80,98.19,142391.0
2011-12-22 10:40:00-08:00,99.73,99.85,100.05,98.51,99.21,92400.0
2011-12-23 10:40:00-08:00,99.69,99.73,100.22,99.56,99.98,51457.0
2011-12-27 10:40:00-08:00,101.20,101.20,101.39,99.49,99.51,59211.0
2011-12-28 10:40:00-08:00,99.70,99.60,101.50,99.11,101.02,95264.0
2011-12-29 10:40:00-08:00,99.68,99.61,99.92,98.30,99.55,91185.0
2011-12-30 10:40:00-08:00,99.72,99.71,100.11,98.61,99.33,70551.0


In [40]:
new_series.c.sum()

2392.73

In [18]:
prev_prices = prev_series.loc[prev_series.index[-1]]
new_prices = new_series.loc[prev_series.index[-1]]

In [19]:
prev_prices

c          100.64
exec       100.64
h          102.06
l           99.84
o          100.66
v       154867.00
Name: 2012-01-18 10:40:00-08:00, dtype: float64

In [20]:
new_prices

c         100.79
exec      100.79
h         102.24
l         100.00
o         100.82
v       81482.00
Name: 2012-01-18 10:40:00-08:00, dtype: float64

In [21]:
fut_offset = new_prices['exec'] - prev_prices['exec']

In [22]:
fut_offset

0.15000000000000568

In [23]:
new_series[['o','h','l', 'c', 'exec']] -= fut_offset

In [24]:
new_series = new_series[new_series.index > prev_series.index[-1]]

In [ ]:
pd.concat([prev_series, new_series])

In [ ]:
def merge(prev_series, new_series):
    try:
        prev_prices = prev_series.loc[prev_series.index[-1]]
        new_prices = new_series.loc[prev_series.index[-1]]
        # Calculating futures rollover factor
        fut_offset = new_prices['exec'] - prev_prices['exec']
    except KeyError:
        fut_offset = 0.0
        
    new_series[['o','h','l', 'c', 'exec']] -= fut_offset
    new_series = new_series[new_series.index > prev_series.index[-1]]
    return pd.concat([prev_series, new_series])

In [ ]:
%timeit compress_daily(DataFrameGetter(series), fut)

In [ ]:
prev_series, position1  = compress_daily(DataFrameGetter(series), fut)
new_series, position2  = compress_daily(DataFrameGetter(series2), fut2)

In [ ]:
%lprun -f merge merge(prev_series, new_series)

In [ ]:
%timeit merge(prev_series, new_series)

In [ ]:
position1.at[position1.index[-1], 'qty'] = 0

In [ ]:
position1

In [ ]:
position2

In [ ]:
%%timeit
pd.merge(position2.reset_index(), position1.reset_index(), how='outer').sort_values('date').set_index(['date', 'asset'])

In [ ]:
position1.index.get_level_values('date')

In [ ]:
from tmqrfeed.quotes.quote_contfut import QuoteContFut

In [ ]:
qcont_fut = QuoteContFut('US.CL', datafeed=feed, timeframe='D')

In [ ]:
%lprun -m tmqrfeed.quotes.quote_contfut qcont_fut.build()

In [ ]:
%timeit qcont_fut.build()

In [ ]:
%timeit compress_daily(DataFrameGetter(series), fut)